import pandas as pd

In [1]:
import pandas as pd
import glob

In [7]:
path = r'C:\Users\Dell\Desktop\cluster 12'
filenames = glob.glob(path + "/*.csv")

dfs = []

for filename in filenames:

    dfs.append(pd.read_csv(filename))

# Concatenate all data into one DataFrame

tripadv_review = pd.concat(dfs, ignore_index=True)
tripadv_review

,r_points,r_text,place
0,40,Great collection of paintings and artifacts fr...,A G Leventis Gallery
1,50,This is a relatively new gallery of one man's ...,A G Leventis Gallery
2,40,First time that I have been extremely impresse...,A G Leventis Gallery
3,50,"Very new museum, artefacts range from Medieval...",A G Leventis Gallery
4,50,"Ecellent vernisage of belgian Painters, with m...",A G Leventis Gallery
...,...,...,...
10627,50,This is probably one of the best eateries in o...,Zanettos
10628,50,There exist only one menu composed from tradit...,Zanettos
10629,50,Been there with friends and had the full meze....,Zanettos
10630,50,One of the oldest surviving restaurants in Nic...,Zanettos


In [8]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [9]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [11]:
processed_reviews = tripadv_review['r_text'].astype(str).map(preprocess)
processed_reviews

0        [great, collection, paintings, artifacts, leve...
1        [relatively, gallery, collection, paintings, c...
2        [time, extremely, impressed, write, collection...
3        [museum, artefacts, range, medieval, impressio...
4        [ecellent, vernisage, belgian, painters, know,...
                               ...                        
10627    [probably, best, eateries, nicosia, thing, nee...
10628    [exist, menu, composed, traditional, meze, sel...
10629    [friends, meze, food, good, night, couldn, bre...
10630    [oldest, surviving, restaurants, nicosia, appa...
10631    [griiled, food, local, drinks, smiling, staff,...
Name: r_text, Length: 10632, dtype: object

In [12]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]]


In [18]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=11000,
                                           passes=150,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 800,
                                           eval_every = None)

In [19]:
from pprint import pprint

In [20]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.113*"fish" + 0.035*"fresh" + 0.029*"best" + 0.027*"meze" + '
  '0.026*"quality" + 0.024*"service" + 0.023*"sushi" + 0.021*"restaurant" + '
  '0.020*"good" + 0.018*"excellent"'),
 (1,
  '0.024*"chicken" + 0.022*"salad" + 0.019*"meat" + 0.015*"platter" + '
  '0.014*"ordered" + 0.013*"souvlaki" + 0.012*"pork" + 0.012*"restaurant" + '
  '0.012*"grilled" + 0.011*"delicious"'),
 (2,
  '0.011*"hair" + 0.009*"pictures" + 0.006*"importance" + 0.004*"having" + '
  '0.004*"kipros" + 0.003*"admire" + 0.003*"helped" + 0.003*"human" + '
  '0.003*"open" + 0.003*"white"'),
 (3,
  '0.006*"discount" + 0.005*"satisfied" + 0.004*"alternative" + 0.004*"plater" '
  '+ 0.003*"hospitality" + 0.003*"caring" + 0.003*"knowing" + 0.003*"stay" + '
  '0.003*"sufficient" + 0.003*"past"'),
 (4,
  '0.013*"restaurant" + 0.013*"like" + 0.012*"meze" + 0.011*"cypriot" + '
  '0.009*"people" + 0.009*"outside" + 0.008*"table" + 0.008*"dishes" + '
  '0.007*"zanettos" + 0.007*"cyprus"'),
 (5,
  '0.037*"turkish" + 0.0

In [21]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19     0.263552 -0.090214       1        1  24.694672
10     0.126466  0.077551       2        1  17.073629
15     0.104512 -0.113486       3        1   8.302625
5      0.065489  0.232094       4        1   7.259667
1      0.146229 -0.146580       5        1   7.178392
7      0.106908  0.219066       6        1   6.084651
11     0.040849  0.211870       7        1   4.583822
6      0.149757 -0.121831       8        1   4.545325
8      0.101428  0.011532       9        1   3.269780
0      0.126270 -0.161410      10        1   3.024901
16     0.016346  0.205029      11        1   2.820041
4     -0.011460 -0.095107      12        1   2.479732
14    -0.128666 -0.035921      13        1   2.049875
13     0.006401 -0.040657      14        1   1.731342
18    -0.112616  0.001219      15        1   1.653371
12    -0.102706 -0.045943      16        1   0.973269
9     -0.215598 -0.006118      17        1   0.685314
17    -0.213310 -0.046191      18        1   0.614341
2     -0.235302 -0.017305      19        1   0.538440
3     -0.234549 -0.037595      20        1   0.436809, topic_info=           Term         Freq        Total Category  logprob  loglift
112        food  5579.000000  5579.000000  Default  30.0000  30.0000
183       hotel  2338.000000  2338.000000  Default  29.0000  29.0000
131        good  5260.000000  5260.000000  Default  28.0000  28.0000
1798       fish   989.000000   989.000000  Default  27.0000  27.0000
461        room  1592.000000  1592.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
194      walked     3.351336   100.253998  Topic20  -5.9335   2.0351
867      couldn     3.486518   157.024811  Topic20  -5.8940   1.6259
1809  welcoming     3.348673   154.345306  Topic20  -5.9343   1.6028
25         know     3.352405   289.503021  Topic20  -5.9332   0.9749
574        wait     3.353860   321.342804  Topic20  -5.9327   0.8710

[1466 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
3593      4  0.939787  abandoned
3593     20  0.039158  abandoned
6169     18  0.434060      abuse
1983      4  0.361855     accept
1983      7  0.072371     accept
...     ...       ...        ...
3694      7  0.021343       zone
3694     11  0.014229       zone
3694     12  0.021343       zone
3694     15  0.007114       zone
3694     20  0.007114       zone

[4346 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 11, 16, 6, 2, 8, 12, 7, 9, 1, 17, 5, 15, 14, 19, 13, 10, 18, 3, 4])